# 1 创建坐标张量

In [2]:
import torch

height = 2
width = 2

coords_h = torch.arange(height)
coords_w = torch.arange(width)
coords = torch.stack(torch.meshgrid([coords_h, coords_w]))
coords

e:\Apps\anaconda3\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([[[0, 0],
         [1, 1]],

        [[0, 1],
         [0, 1]]])

# 2 展平

In [3]:
coords_flatten = torch.flatten(coords, 1)
coords_flatten

tensor([[0, 0, 1, 1],
        [0, 1, 0, 1]])

上述结果，生成了形状为 2x4 的张量，
- 第一行表示高度坐标
- 第二行表示宽度坐标

# 3 增加维度
- 对于高度方向，在最后增加一个新维度，使形状从 `[2, 4]` -> `[2, 4, 1]`
- 对于宽度方向，在中间增加一个新维度，使形状从 `[2, 4]` -> `[2, 1, 4]`

In [4]:
coords_flatten_1 = coords_flatten[:,:, None]
coords_flatten_2 = coords_flatten[:, None,:]

In [5]:
coords_flatten_1

tensor([[[0],
         [0],
         [1],
         [1]],

        [[0],
         [1],
         [0],
         [1]]])

In [6]:
coords_flatten_2

tensor([[[0, 0, 1, 1]],

        [[0, 1, 0, 1]]])

# 4 计算相对坐标差
对坐标进行广播
- 对coords_flatten_1 张量的形状进行扩展，从 `[2, 4, 1]` -> `[2, 4, 4]`
- 对coords_flatten_2 张量的形状进行扩展，从 `[2, 1, 4]` -> `[2, 4, 4]`

In [7]:
coords_flatten_1_broadcast = coords_flatten_1.expand(-1, -1, 4)
coords_flatten_2_broadcast = coords_flatten_2.expand(-1, 4, -1)

In [8]:
coords_flatten_1_broadcast

tensor([[[0, 0, 0, 0],
         [0, 0, 0, 0],
         [1, 1, 1, 1],
         [1, 1, 1, 1]],

        [[0, 0, 0, 0],
         [1, 1, 1, 1],
         [0, 0, 0, 0],
         [1, 1, 1, 1]]])

In [9]:
coords_flatten_2_broadcast

tensor([[[0, 0, 1, 1],
         [0, 0, 1, 1],
         [0, 0, 1, 1],
         [0, 0, 1, 1]],

        [[0, 1, 0, 1],
         [0, 1, 0, 1],
         [0, 1, 0, 1],
         [0, 1, 0, 1]]])

观察上述结果
coords_flatten_1_broadcast 的结果显示，它是一个[2, 4, 4]的张量，
- `[0,:,:]`表示高度方向上的坐标，在水平方向上进行扩展，每一行中的元素表示相同的位置的高度
- `[1:,:,:]`表示宽度方向上的坐标，在水平方向上进行扩展，每一行中的元素表示相同的位置的宽度


coords_flatten_2_broadcast 的结果显示，它是一个[2, 4, 4]的张量，
- `[0,:,:]`表示高度方向上的坐标，在竖直方向上进行扩展，每一行中的元素表示不同的位置的高度
- `[1:,:,:]`表示宽度方向上的坐标，在竖直方向上进行扩展，每一行中的元素表示不同的位置的宽度

计算坐标相对位置差的思想是：
每个坐标位置点与其他所有坐标位置坐标点的差值，即为相对位置差。

因此，我们可以计算相对位置差的公式如下：

relative_position_diff = coords_flatten_1_broadcast - coords_flatten_2_broadcast


In [10]:
relatetive_coords = coords_flatten_1_broadcast - coords_flatten_2_broadcast

In [11]:
relatetive_coords

tensor([[[ 0,  0, -1, -1],
         [ 0,  0, -1, -1],
         [ 1,  1,  0,  0],
         [ 1,  1,  0,  0]],

        [[ 0, -1,  0, -1],
         [ 1,  0,  1,  0],
         [ 0, -1,  0, -1],
         [ 1,  0,  1,  0]]])

relative_coords 结果是`[2, 4, 4]`
- `[0,:,:]` 表示高度方向上的坐标差，`[0, i, j]`表示窗口内第`i`个点相对于第`j`个点的坐标高度方向上的坐标差。

第一行表示`(0,0)`点到其他所有坐标点的高度方向上的差值；

第二行表示`(0,1)`点到其他所有坐标点的高度方向上的差值；

第三行表示`(1,0)`点到其他所有坐标点的高度方向上的差值；

第四行表示`(1,1)`点到其他所有坐标点的高度方向上的差值。

例如：`[0,0,0]` 表示 `(0,0)` 点到`(0,0)`的高度方向上的差值, 值为0。

`[0,0,1]`表示`(0,0)`点到`(0,1)`的高度方向上的差值, 值为0。

`[0,1,0]`表示`(0,1)`点到`(0,0)`的高度方向上的差值, 值为0。

`[0,1,3]`表示`(0,1)`点到`(1,1)`的高度方向上的差值, 值为-1。

---

- `[1,:,:]` 表示宽度方向上的坐标差，`[1, i, j]`表示窗口内第`i`个点相对于第`j`个点的坐标宽度方向上的坐标差。

第一行表示`(0,0)`点到其他所有坐标点的宽度方向上的差值；

第二行表示`(0,1)`点到其他所有坐标点的宽度方向上的差值；

第三行表示`(1,0)`点到其他所有坐标点的宽度方向上的差值；

第四行表示`(1,1)`点到其他所有坐标点的宽度方向上的差值。

例如：`[1,0,0]` 表示 `(0,0)` 点到`(0,0)`的宽度方向上的差值, 值为0。

`[1,0,1]`表示`(0,0)`点到`(0,1)`的宽度方向上的差值, 值为-1。        

`[1,1,0]`表示`(0,1)`点到`(0,0)`的宽度方向上的差值, 值为1。

`[1,1,3]`表示`(0,1)`点到`(1,1)`的宽度方向上的差值, 值为0。



# 5 将相对坐标差转换为三维张量

In [12]:
relatetive_coords = relatetive_coords.permute(1, 2, 0).contiguous()

In [13]:
relatetive_coords

tensor([[[ 0,  0],
         [ 0, -1],
         [-1,  0],
         [-1, -1]],

        [[ 0,  1],
         [ 0,  0],
         [-1,  1],
         [-1,  0]],

        [[ 1,  0],
         [ 1, -1],
         [ 0,  0],
         [ 0, -1]],

        [[ 1,  1],
         [ 1,  0],
         [ 0,  1],
         [ 0,  0]]])

上述结果是一个形状为`[4,4,2]`的张量，`[i,j,:]`表示第i+1个坐标点相对第j+1个坐标点的坐标差值。

`[0,:,:]`表示第1个坐标点相对其他所有坐标点的坐标差值，

`[1,:,:]`表示第2个坐标点相对其他所有坐标点的坐标差值，

以此类推，`[3,:,:]`表示第4个坐标点相对其他所有坐标点的坐标差值。

例如：

`[0,0,:]`表示第1个坐标点相对第1个坐标点的坐标差值，`[0,0]`，表示第1个坐标点和第1个坐标点（自己）的高度差为0，宽度差为0。

`[0,1,:]`表示第1个坐标点相对第2个坐标点的坐标差值，`[0,1]`，表示第1个坐标点相对第2个坐标点的高度差为0，宽度差为-1。

`[1,2,:]`表示第2个坐标点相对第3个坐标点的坐标差值，`[-1,1]`，表示第2个坐标点相对第3个坐标点的高度差为-1，宽度差为1。

以此类推，`[3,3,:]`表示第4个坐标点相对第4个坐标点的坐标差值，`[0,0]`，表示第4个坐标点和第4个坐标点（自己）的高度差为0，宽度差为0。